# OPTAA Checker

This notebook is designed to check the SPKIR csv calibration file in pull request. The process I follow is:
1. Read in the OPTAA csv from the pull request into a pandas dataframe
2. Identify the source file of the calibration coefficients
3. Parse the calibration coefficients directly from the source file
4. Compare the OPTAA csv from the pull request with the csv parsed from the source file

**====================================================================================================================**

The first step is to load relevant packages:

In [1]:
import csv
import re
import os
import shutil
import numpy as np
import pandas as pd

In [2]:
from utils import *

In [3]:
from zipfile import ZipFile
import string

**====================================================================================================================**
Define the directories where the **csv** file to check is stored, and where the **source** file is stored. Make sure to check the following information on your machine via your terminal first:
1. The branch of your local asset-management repository matches the location of the OPTAA cals.
2. Your local asset-management repository has the requisite **csv** file to check
3. You have downloaded the **source** of the csv file

In [4]:
csv_dir = '/home/andrew/Documents/OOI-CGSN/asset-management/calibration/OPTAAD/'
source_dir = '/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/OPTAA/OPTAA_Cal/'

**====================================================================================================================**
### Find & Parse the source file
Now, we want to find the source file of the calibration coefficients, parse the data using the optaa parser, and read the data into a pandas dataframe. The key pieces of data needed for the parser are:
1. Instrument UID: This is needed to initialize the OPTAA parser
2. Source file: This is the full path to the source file. Zip files are acceptable input.

In [238]:
source_name = '129'
for file in os.listdir(source_dir):
    if source_name in file:
        source_file = file
        print(source_file)

OPTAA-D_AC-S_SN_129_Calibration_Files_2015-09-30.zip
OPTAA-D_AC-S_SN_129_Calibration_Files_2015-10-05.zip
OPTAA-D_AC-S_SN_129_Calibration_Files_2017-02-09.zip
OPTAA-D_AC-S_SN_129_Calibration_Files_2018-04-11.zip
OPTAA-D_AC-S_SN_129_Calibration_Sheet_2013-01-30.pdf
OPTAA-D_AC-S_SN_129_Calibration_Files_2013-01-30.zip
OPTAA-D_AC-S_SN_129_Calibration_Files_2013-01-23.zip


In [239]:
source_file = 'OPTAA-D_AC-S_SN_129_Calibration_Files_2018-04-11.zip'

Initialize the parser:

In [240]:
optaa = OPTAACalibration('CGINS-OPTAAD-00129')

Read in the calibration coefficients:

In [241]:
optaa.load_cal(source_dir+source_file)

Write the csv to a temporary local folder:

In [242]:
temp_directory = '/'.join((os.getcwd(),'temp'))
# Check if the temp directory exists; if it already does, purge and rewrite
if os.path.exists(temp_directory):
    shutil.rmtree(temp_directory)
    ensure_dir(temp_directory)
else:
    ensure_dir(temp_directory)

In [243]:
optaa.write_csv(temp_directory)

Write CGINS-OPTAAD-00129__20180411.csv to /home/andrew/Documents/OOI-CGSN/QAQC_Sandbox/Metadata_Review/temp? [y/n]: y


In [244]:
os.listdir(temp_directory)

['CGINS-OPTAAD-00129__20180411.csv',
 'CGINS-OPTAAD-00129__20180411__CC_tcarray.ext',
 'CGINS-OPTAAD-00129__20180411__CC_taarray.ext']

In [245]:
optaa.uid, optaa.serial, optaa.date

('CGINS-OPTAAD-00129', 'ACS-129', '20180411')

**====================================================================================================================**
### Check the data
Now, we have generated local csv and ext files from the data. We can now reload that data into python as a pandas dataframe, which will allow for a direct comparison with the existing data. 

In [246]:
sn = optaa.serial.split('-')[1].zfill(5)
dt = optaa.date

In [247]:
source_csv = pd.read_csv(temp_directory+'/CGINS-OPTAAD-'+sn+'__'+dt+'.csv')
source_csv

,serial,name,value,notes
0,ACS-129,CC_acwo,"[0.872111, 0.993926, 1.056039, 1.078307, 1.077...",Source file: OPTAA-D_AC-S_SN_129_Calibration_...
1,ACS-129,CC_awlngth,"[401.9, 406.2, 410.3, 414.4, 418.7, 423.7, 428...",NaN
2,ACS-129,CC_ccwo,"[-0.350739, -0.218253, -0.110413, -0.019578, 0...",NaN
3,ACS-129,CC_cwlngth,"[401.6, 406.2, 410.3, 414.4, 418.9, 423.8, 428...",NaN
4,ACS-129,CC_taarray,SheetRef:CC_taarray,NaN
5,ACS-129,CC_tbins,"[0.838339, 1.379098, 2.454757, 3.467857, 4.477...",NaN
6,ACS-129,CC_tcal,19.4,NaN
7,ACS-129,CC_tcarray,SheetRef:CC_tcarray,NaN


In [248]:
source_csv['notes'].iloc[0]

' Source file: OPTAA-D_AC-S_SN_129_Calibration_Files_2018-04-11.zip > acs129.dev'

In [249]:
source_taarray = pd.read_csv(temp_directory+'/CGINS-OPTAAD-'+sn+'__'+dt+'__CC_taarray.ext',header=None)
source_taarray.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,-0.174987,-0.175900,-0.168615,-0.160970,-0.152376,-0.144623,-0.136350,-0.126454,-0.117194,-0.108134,...,0.003770,0.008267,0.013676,0.018307,0.023222,0.027569,0.031567,0.036784,0.040841,0.042694
1,-0.159983,-0.160125,-0.152900,-0.145823,-0.137779,-0.130839,-0.122764,-0.113213,-0.105074,-0.096508,...,0.003455,0.007021,0.011235,0.015005,0.019185,0.022829,0.025822,0.030262,0.032911,0.033513
2,-0.144986,-0.144293,-0.137254,-0.130806,-0.123771,-0.117132,-0.110258,-0.101502,-0.093412,-0.085621,...,0.002464,0.005652,0.008833,0.011920,0.015239,0.017843,0.020387,0.023302,0.025853,0.026244
3,-0.131093,-0.129925,-0.123696,-0.117741,-0.111044,-0.105054,-0.098471,-0.090473,-0.083096,-0.075384,...,0.002002,0.004650,0.007270,0.009898,0.012539,0.014473,0.016355,0.018872,0.020424,0.021104
4,-0.121099,-0.119726,-0.113708,-0.108367,-0.102094,-0.096773,-0.090682,-0.083108,-0.076016,-0.069058,...,0.001321,0.003752,0.006063,0.007939,0.010129,0.011793,0.012766,0.014842,0.016205,0.016511


In [250]:
source_tcarray = pd.read_csv(temp_directory+'/CGINS-OPTAAD-'+sn+'__'+dt+'__CC_tcarray.ext',header=None)
source_tcarray.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,0.018697,0.021365,0.022024,0.021488,0.020540,0.019812,0.019234,0.017540,0.017627,0.016834,...,-0.001127,-0.002519,-0.003310,-0.004145,-0.005943,-0.005522,-0.007214,-0.008587,-0.009662,-0.004302
1,0.014328,0.016853,0.017678,0.017287,0.016439,0.015821,0.015683,0.015867,0.014792,0.014884,...,-0.001356,-0.001710,-0.001514,-0.002555,-0.002436,-0.002763,-0.004943,-0.004963,-0.005987,-0.010034
2,0.007788,0.010362,0.011731,0.011253,0.010670,0.010156,0.009944,0.010192,0.009716,0.008756,...,-0.000495,-0.001022,-0.001217,-0.001755,-0.001672,-0.001953,-0.003689,-0.004020,-0.004888,-0.007349
3,0.002893,0.005807,0.006628,0.007196,0.006545,0.006126,0.006809,0.006559,0.006565,0.006653,...,-0.000538,-0.001630,-0.000965,-0.001164,-0.001612,-0.001559,-0.002644,-0.002619,-0.003860,-0.002004
4,0.000621,0.003448,0.004762,0.004388,0.004496,0.004356,0.004613,0.004832,0.005041,0.005009,...,-0.001095,-0.000674,-0.000251,-0.001230,-0.001090,-0.000817,-0.002250,-0.002033,-0.002713,-0.004786


**====================================================================================================================**
# OPTAA Parser
Below is a parser for the OPTAA calibration file. The following methods are available as part of the OPTAACalibration class:
* **OPTAACalibration.load_cal**:
        
         Wrapper function to load all of the calibration coefficients
        
         Args:
            filepath - path to the directory with filename which has the
                calibration coefficients to be parsed and loaded
         Calls:
            open_cal
            parse_cal
            
* **OPTAACalibration.load_qct**:

        Wrapper function to load the calibration coefficients from
        the QCT checkin.
            

It is used as follows:
1. Initialize the OPTAA class using the **UID** for the OPTAA with the following code: OPTAA = OPTAACalibration(UID)
2. 

In [8]:
from zipfile import ZipFile
class OPTAACalibration():
    # Class that stores calibration values for CTDs.

    def __init__(self, uid):
        self.serial = None
        self.nbins = None
        self.uid = uid
        self.sigfig = 6
        self.date = []
        self.coefficients = {
            'CC_acwo': [],
            'CC_awlngth': [],
            'CC_ccwo': [],
            'CC_cwlngth': [],
            'CC_taarray': 'SheetRef:CC_taarray',
            'CC_tbins': [],
            'CC_tcal': [],
            'CC_tcarray': 'SheetRef:CC_tcarray'
        }
        self.tcarray = []
        self.taarray = []
        self.notes = {
            'CC_acwo': '',
            'CC_awlngth': '',
            'CC_ccwo': '',
            'CC_cwlngth': '',
            'CC_taarray': '',
            'CC_tbins': '',
            'CC_tcal': '',
            'CC_taarray': ''
        }

    @property
    def uid(self):
        return self._uid

    @uid.setter
    def uid(self, d):
        r = re.compile('.{5}-.{6}-.{5}')
        if r.match(d) is not None:
            self._uid = d
            serial = d.split('-')[-1].lstrip('0')
            self.serial = 'ACS-' + serial
        else:
            raise Exception(f"The instrument uid {d} is not a valid uid. Please check.")

            
    def load_cal(self, filepath):
        """
        Wrapper function to load all of the calibration coefficients
        
        Args:
            filepath - path to the directory with filename which has the
                calibration coefficients to be parsed and loaded
        Calls:
            open_cal
            parse_cal
        """
        
        data = self.open_dev(filepath)
        
        self.parse_dev(data)
        
        
    def load_qct(self, filepath):
        """
        Wrapper function to load the calibration coefficients from
        the QCT checkin.
        """
        
        data = self.open_dev(filepath)
        
        self.parse_qct(data)
    
    
    def open_dev(self, filepath):
        """
        Function that opens and reads in cal file
        information for a OPTAA. Zipfiles are acceptable inputs.
        """
        
        if filepath.endswith('.zip'):
            with ZipFile(filepath) as zfile:
                # Check if OPTAA has the .dev file
                filename = [name for name in zfile.namelist() if name.lower().endswith('.dev')]
                
                # Get and open the latest calibration file
                if len(filename) == 1:
                    data = zfile.read(filename[0]).decode('ascii')
                    self.source_file(filepath, filename[0])
                    
                elif len(filename) > 1:
                    raise FileExistsError(f"Multiple .dev files found in {filepath}.")

                else:
                    raise FileNotFoundError(f"No .dev file found in {filepath}.")
                        
        elif filepath.lower().endswith('.dev'):
            with open(filepath) as file:
                data = file.read()
            self.source_file(filepath, file)
                
        elif filepath.lower().endswith('.dat'):
            with open(filepath) as file:
                data = file.read()
            self.source_file(filepath, file)
            
        else:
            raise FileNotFoundError(f"No .dev file found in {filepath}.")
        
        return data


    def source_file(self, filepath, filename):
        """
        Routine which parses out the source file and filename
        where the calibration coefficients are sourced from.
        """
        
        if filepath.lower().endswith('.dev'):
            dcn = filepath.split('/')[-2]
            filename = filepath.split('/')[-1]
        else:
            dcn = filepath.split('/')[-1]
        
        self.source = f'Source file: {dcn} > {filename}'
        

    def parse_dev(self, data):
        """
        Function to parse the .dev file in order to load the
        calibration coefficients for the OPTAA.
        
        Args:
            data - opened .dev file in ascii-format
        """
        
        for line in data.splitlines():
            # Split the data based on data -> header split
            parts = line.split(';')
                # If the len isn't number 2, 
            if len(parts) is not 2:
                # Find the calibration temperature and date
                if 'tcal' in line.lower():
                    line = ''.join((x for x in line if x not in [y for y in string.punctuation if y is not '/']))
                    parts = line.split()
                    # Calibration temperature
                    tcal = parts[1].replace('C','')
                    tcal = float(tcal)/10
                    self.coefficients['CC_tcal'] = tcal
                    # Calibration date
                    date = parts[-1].strip(string.punctuation)
                    self.date = pd.to_datetime(date).strftime('%Y%m%d')
        
            else:
                info, comment = parts
                
                if comment.strip().startswith('temperature bins'):
                    tbins = [float(x) for x in info.split()]
                    self.coefficients['CC_tbins'] = tbins
                    
                elif comment.strip().startswith('number'):
                    self.nbins = int(float(info.strip()))
                    
                elif comment.strip().startswith('C'):
                    if self.nbins is None:
                        raise AttributeError(f'Failed to load number of temperature bins.')
                        
                    # Parse out the different calibration coefficients
                    parts = info.split()
                    cwlngth = float(parts[0][1:])
                    awlngth = float(parts[1][1:])
                    ccwo = float(parts[3])
                    acwo = float(parts[4])
                    tcrow = [float(x) for x in parts[5:self.nbins+5]]
                    acrow = [float(x) for x in parts[self.nbins+5:2*self.nbins+5]]
                
                    # Now put the coefficients into the coefficients dictionary
                    self.coefficients['CC_acwo'].append(acwo)
                    self.coefficients['CC_awlngth'].append(awlngth)
                    self.coefficients['CC_ccwo'].append(ccwo)
                    self.coefficients['CC_cwlngth'].append(cwlngth)
                    self.tcarray.append(tcrow)
                    self.taarray.append(acrow)
                    
                    
    def parse_qct(self, data):
        """
        This function is designed to parse the QCT file, which contains the
        calibration data in slightly different format than the .dev file
        
        
        """
        
        for line in data.splitlines():
            if 'WetView' in line:
                _, _, _, date, time = line.split()
                try:
                    date_time = date + ' ' + time
                    self.date = pd.to_datetime(date_time).strftime('%Y%m%d')
                except:
                    date_time = from_excel_ordinal(float(date) + float(time))
                    self.date = pd.to_datetime(date_time).strftime('%Y%m%d')
                continue
                
            parts = line.split(';')
            
            if len(parts) == 2:
                if comment.strip().startswith('temperature bins'):
                    tbins = [float(x) for x in info.split()]
                    self.coefficients['CC_tbins'] = tbins
                    
                elif comment.strip().startswith('number'):
                    self.nbins = int(float(info.strip()))
                    
                elif comment.strip().startswith('C'):
                    if self.nbins is None:
                        raise AttributeError(f'Failed to load number of temperature bins.')
                    # Parse out the different calibration coefficients
                    parts = info.split()
                    cwlngth = float(parts[0][1:])
                    awlngth = float(parts[1][1:])
                    ccwo = float(parts[3])
                    acwo = float(parts[4])
                    tcrow = [float(x) for x in parts[5:self.nbins+5]]
                    acrow = [float(x) for x in parts[self.nbins+5:(2*self.nbins)+5]]
                    
                    # Now put the coefficients into the coefficients dictionary
                    self.coefficients['CC_acwo'].append(acwo)
                    self.coefficients['CC_awlngth'].append(awlngth)
                    self.coefficients['CC_ccwo'].append(ccwo)
                    self.coefficients['CC_cwlngth'].append(cwlngth)
                    self.tcarray.append(tcrow)
                    self.taarray.append(acrow)                
    
                        
    def write_csv(self, outpath):
        """
        This function writes the correctly named csv file for the ctd to the
        specified directory.

        Args:
            outpath - directory path of where to write the csv file
        Raises:
            ValueError - raised if the CTD object's coefficient dictionary
                has not been populated
        Returns:
            self.to_csv - a csv of the calibration coefficients which is
                written to the specified directory from the outpath.
        """

        # Run a check that the coefficients have actually been loaded
        if len(self.coefficients.values()) <= 2:
            raise ValueError('No calibration coefficients have been loaded.')

        # Create a dataframe to write to the csv
        data = {
            'serial': [self.serial]*len(self.coefficients),
            'name': list(self.coefficients.keys()),
            'value': list(self.coefficients.values())
        }
        df = pd.DataFrame().from_dict(data)
      
        # Now merge the coefficients dataframe with the notes
        notes = pd.DataFrame().from_dict({
            'name':list(self.notes.keys()),
            'notes':list(self.notes.values())
        })
        df = df.merge(notes, how='outer', left_on='name', right_on='name')
            
        # Add in the source file
        df['notes'].iloc[0] = df['notes'].iloc[0] + ' ' + self.source
        
        # Sort the data by the coefficient name
        df = df.sort_values(by='name')

        # Generate the csv names
        csv_name = self.uid + '__' + self.date + '.csv'
        tca_name = self.uid + '__' + self.date + '__' + 'CC_tcarray.ext'
        taa_name = self.uid + '__' + self.date + '__' + 'CC_taarray.ext'
        
        def write_array(filename, cal_array):
            with open(filename, 'w') as out:
                array_writer = csv.writer(out)
                array_writer.writerows(cal_array)

        # Write the dataframe to a csv file
        check = input(f"Write {csv_name} to {outpath}? [y/n]: ")
        # check = 'y'
        if check.lower().strip() == 'y':
            df.to_csv(outpath+'/'+csv_name, index=False)
            write_array(outpath+'/'+tca_name, self.tcarray)
            write_array(outpath+'/'+taa_name, self.taarray)